# Logistic Regression

In [1]:
import pandas as pd
import numpy as np
# imports from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold

lg = LogisticRegression()

In [2]:
#loading in the datasets
df_training = pd.read_csv('../data/archive/train_pca.csv')
df_testing = pd.read_csv('../data/archive/test_pca.csv')

In [3]:
x_train = df_training.iloc[:, :-1]
y_train = df_training.iloc[:,-1]

x_test = df_testing.iloc[:, :-1]
y_test = df_testing.iloc[:,-1]

In [4]:
#Making a grid of values we want our grid search to test to find the best parameters

grid_values = [{'penalty': ['l2'], 'C' :[.01,.05,.1,.3,.5,.8,1], 'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 'multi_class': ['multinomial'], 'max_iter' : [2000] }]


lg_classifier = GridSearchCV(estimator = lg, param_grid = grid_values, cv = StratifiedKFold(n_splits = 5),
                      scoring = ['accuracy', 'roc_auc_ovr', 'f1_micro'], refit = False, verbose = 0)

lg_model = lg_classifier.fit(x_train, y_train)

In [5]:
#initializing our lists so we can later store the best params that achieve best scores across metrics
accuracy_best_params = []
precision_best_params = []
f1_best_params = []

accuracy_best_params.append(lg_model.cv_results_['params'][ np.argmin(lg_model.cv_results_['rank_test_accuracy'])])
precision_best_params.append(lg_model.cv_results_['params'][ np.argmin(lg_model.cv_results_['rank_test_roc_auc_ovr'])])
f1_best_params.append(lg_model.cv_results_['params'][ np.argmin(lg_model.cv_results_['rank_test_f1_micro']) ])

In [6]:
accuracy_best_params

[{'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'newton-cg'}]

In [7]:
precision_best_params

[{'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'newton-cg'}]

In [8]:
f1_best_params

[{'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'newton-cg'}]

In [9]:
#finding the accuracy metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score

    
def accuracy_metric(xtrain,ytrain,xtest,ytest):
    accuracy_test_score = []
    dt_clf = LogisticRegression(C=accuracy_best_params[0]['C'], 
                                         multi_class = accuracy_best_params[0]['multi_class'], 
                                         penalty = accuracy_best_params[0]['penalty'],
                                         solver = accuracy_best_params[0]['solver'],
                                         max_iter = accuracy_best_params[0]['max_iter'])
    model = dt_clf.fit(xtrain, ytrain)
    y_pred = model.predict(xtest)
    accuracy_test_score.append(accuracy_score(ytest, y_pred))
    return accuracy_test_score

def f1_metric(xtrain,ytrain,xtest,ytest):
    f1_test_score = []
    dt_clf = LogisticRegression(C=f1_best_params[0]['C'], 
                                         multi_class = f1_best_params[0]['multi_class'], 
                                         penalty = f1_best_params[0]['penalty'],
                                         solver = f1_best_params[0]['solver'],
                                         max_iter = f1_best_params[0]['max_iter'])
    model = dt_clf.fit(xtrain, ytrain)
    y_pred = model.predict(xtest)
    f1_test_score.append(f1_score(ytest, y_pred,average = 'micro'))
    return f1_test_score

def precision_metric(xtrain,ytrain,xtest,ytest):
    precision_test_score = []
    dt_clf = LogisticRegression(C=precision_best_params[0]['C'], 
                                         multi_class = precision_best_params[0]['multi_class'], 
                                         penalty = precision_best_params[0]['penalty'],
                                         solver = precision_best_params[0]['solver'],
                                         max_iter = precision_best_params[0]['max_iter'])
    model = dt_clf.fit(xtrain, ytrain)
    y_pred = model.predict(xtest)
    precision_test_score.append(precision_score(ytest, y_pred,average = 'micro'))
    return precision_test_score

In [10]:
print(accuracy_metric(x_train,y_train,x_test,y_test))
print(f1_metric(x_train,y_train,x_test,y_test))
print(precision_metric(x_train,y_train,x_test,y_test))

[0.9226331862911435]
[0.9226331862911435]
[0.9226331862911435]


## Bootstrap

In [11]:
#Separating X and y from testing since this won't be used in the for loop
x_train_full = df_training.iloc[:,:-1]
y_train_full = df_training.iloc[:,-1]

x_test = df_testing.iloc[:, :-1]
y_test = df_testing.iloc[:,-1]

In [12]:
#getting all of our randomized samples from the training
all_samples = []

for sample in range(0,5):
    all_samples.append(df_training.sample(n=5000, replace= True))

In [13]:
accuracy_best_params = []
precision_best_params = []
f1_best_params = []


for sample in all_samples:
    
    
    #Making a grid of values we want our grid search to test to find the best parameters
    grid_values = [{'penalty': ['l2'], 'C' :[.01,.05,.1,.3,.5,.8,1], 'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 'multi_class': ['multinomial'], 'max_iter' : [2000] }]


    lg_classifier = GridSearchCV(estimator = lg, param_grid = grid_values, cv = StratifiedKFold(n_splits = 5),
                      scoring = ['accuracy', 'roc_auc_ovr', 'f1_micro'], refit = False, verbose = 0)

    x_train = sample.iloc[:, :-1]
    y_train = sample.iloc[:,-1]
    
    lg_model = lg_classifier.fit(x_train, y_train)
    
    
   

    accuracy_best_params.append(lg_model.cv_results_['params'][ np.argmin(lg_model.cv_results_['rank_test_accuracy'])])
    precision_best_params.append(lg_model.cv_results_['params'][ np.argmin(lg_model.cv_results_['rank_test_roc_auc_ovr'])])
    f1_best_params.append(lg_model.cv_results_['params'][ np.argmin(lg_model.cv_results_['rank_test_f1_micro']) ])
    
    
  

In [14]:
accuracy_best_params

[{'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'sag'},
 {'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'newton-cg'},
 {'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'newton-cg'},
 {'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'newton-cg'},
 {'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'sag'}]

In [15]:
f1_best_params

[{'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'sag'},
 {'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'newton-cg'},
 {'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'newton-cg'},
 {'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'newton-cg'},
 {'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'sag'}]

In [16]:
precision_best_params

[{'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'newton-cg'},
 {'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'newton-cg'},
 {'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'newton-cg'},
 {'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'newton-cg'},
 {'C': 1,
  'max_iter': 2000,
  'multi_class': 'multinomial',
  'penalty': 'l2',
  'solver': 'lbfgs'}]

In [17]:
print(accuracy_metric(x_train,y_train,x_test,y_test))
print(f1_metric(x_train,y_train,x_test,y_test))
print(precision_metric(x_train,y_train,x_test,y_test))

[0.9202578893790295]
[0.9202578893790295]
[0.9205972175093315]


In [18]:
accuracy_test_score = []

i = 0
for param in accuracy_best_params:
 
    dt_clf = LogisticRegression(C=accuracy_best_params[i]['C'], 
                                         multi_class = accuracy_best_params[i]['multi_class'], 
                                         penalty = accuracy_best_params[i]['penalty'],
                                         solver = accuracy_best_params[i]['solver'],
                                         max_iter = accuracy_best_params[i]['max_iter'])
    model = dt_clf.fit(x_train_full, y_train_full)
    y_pred = model.predict(x_test)
    accuracy_test_score.append(accuracy_score(y_test, y_pred))
    i += 1

In [19]:
accuracy_test_score

[0.9222938581608415,
 0.9226331862911435,
 0.9226331862911435,
 0.9226331862911435,
 0.9222938581608415]

In [20]:
f1_test_score = []

i = 0
for param in f1_best_params:
    
    dt_clf = LogisticRegression(C=f1_best_params[0]['C'], 
                                         multi_class = f1_best_params[i]['multi_class'], 
                                         penalty = f1_best_params[i]['penalty'],
                                         solver = f1_best_params[i]['solver'],
                                         max_iter = f1_best_params[i]['max_iter'])
    model = dt_clf.fit(x_train_full, y_train_full)
    y_pred = model.predict(x_test)
    f1_test_score.append(f1_score(y_test, y_pred,average = 'micro'))
    i += 1

In [21]:
f1_test_score

[0.9222938581608415,
 0.9226331862911435,
 0.9226331862911435,
 0.9226331862911435,
 0.9222938581608415]

In [22]:
precision_test_score = []

i = 0
for param in precision_best_params:
    
    dt_clf = LogisticRegression(C=precision_best_params[0]['C'], 
                                         multi_class = precision_best_params[0]['multi_class'], 
                                         penalty = precision_best_params[0]['penalty'],
                                         solver = precision_best_params[0]['solver'],
                                         max_iter = precision_best_params[0]['max_iter'])
    model = dt_clf.fit(x_train_full, y_train_full)
    y_pred = model.predict(x_test)
    precision_test_score.append(precision_score(y_test, y_pred,average = 'micro'))
    i += 1

In [23]:
precision_test_score

[0.9226331862911435,
 0.9226331862911435,
 0.9226331862911435,
 0.9226331862911435,
 0.9226331862911435]